In [1]:
# Basic utilities
import re
import glob
import sys
from datetime import date
from copy import deepcopy
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt

# Data wrangling packages
import numpy as np
from scipy import stats
from skimage.filters import threshold_yen

# Cellpose - the current belle of the ball
from cellpose import models, io

# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Cell Segmentation', 'Imaging Utilities', str(sys.path[0]))

# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
#import ims_file_utils as ims
#import image_processing_utils as impro
from skimage.segmentation import find_boundaries

from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from skimage import io
import glob
from skimage.filters import threshold_local
from skimage.filters import threshold_li
from skimage.color import rgb2gray
from math import sqrt
from skimage.morphology import disk, ball
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.filters.rank import enhance_contrast
from skimage.exposure import adjust_gamma
import pickle as pkl
import sys
from datetime import date
import os, psutil
import re
process = psutil.Process(os.getpid())
# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Registration', 'Imaging Utilities', str(sys.path[0]))
# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
#import ims_file_utils as ims
#import image_processing_utils as impro
#import plotting_funcs as plot
from functools import reduce
from scipy.spatial.distance import cdist, pdist
from skimage.exposure import rescale_intensity
from skimage.segmentation import find_boundaries

# Use this function for local thresholding of images
def lthresh(imgstk):
    th = threshold_local(imgstk, 65, offset=0)
    imgstk = imgstk>= th
    return imgstk
# Use this function to threshold images
def thresh(imgstk):
    th = threshold_li(imgstk)
    imgstk = imgstk>= th
    return imgstk
def spotcall(img, min_sig, max_sig, nsig, th):
    blobs_log = blob_log(img, min_sigma=min_sig, max_sigma=max_sig, num_sigma=nsig, threshold=th)
    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    color = 'lime'
    title = 'Laplacian of Gaussian'
    #plt.rcParams['figure.figsize'] = [20, 15]
    #fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)
    #ax.set_title(title)
    #ax.imshow(img)
    #for blob in blobs_log:
    #    y, x, r = blob
    #    c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
    #    ax.add_patch(c)
    #ax.set_axis_off()
    #plt.tight_layout()
    #plt.show()
    return blobs_log

import cv2
# Basic utilities
from datetime import date
from copy import deepcopy
# Data wrangling packages
from scipy import stats
from skimage.filters import threshold_yen
# Cellpose - the current belle of the ball
from cellpose import models, io
import pandas as pd
from math import sqrt

def numpyToList(points):
    # Takes a tuple of per-axis coordinates and converts it to a list of tuples
    outList = list()
    for i in range(len(points[0])):
        point = list()
        for axis in points:
            point.append(axis[i])
        outList.append(tuple(point))
    return(outList)

def fociDetection(image, masks_nuc, name, outFile):
    # Use laplacian of gaussian to detect blobs
    spots = spotcall(image,min_sig=0.05, max_sig=1,nsig=3,th=0.5) # Decrease threshold if spots are not identified and min sigma if spots are smaller
    spots = np.unique(spots, axis=0)
    print("Num " + name + " foci detected:", len(spots))

    # save spot coordinates
    with open(outFile + '_' + name + '_' + str(len(spots)) + '.pkl', 'wb') as filehandler:
        pkl.dump(spots, filehandler)
        
    return(spots)

In [2]:
def fociDetection(image, masks_nuc, name, filename):
    # Use laplacian of gaussian to detect blobs
    spots = spotcall(image,min_sig=0.05, max_sig=1,nsig=3,th=0.5) # Decrease threshold if spots are not identified and min sigma if spots are smaller
    spots = np.unique(spots, axis=0)
    print("Num " + name + " foci detected:", len(spots))
    return(spots)

## Foci & intensity quantification

In [4]:
runlist= ['P1_1','P1_2','P1_3','P1_4','P1_5','P1_6','P1_7','P1_8','P1_9',
         'P2_1','P2_2','P2_3','P2_4','P2_5', 'P2_6','P2_7','P2_8','P2_9']  
for runName in runlist:
    print(runName)
    opdir = '/mnt/disks/external/jg4159/20231006_DDR364/' # Path to output
    ip = opdir + "RegisteredImages_Ab_" + runName + ".pkl"
    #Loading ims files for different cycles
    mk = opdir + "ref_mem_roundsum_"+runName+".pkl"
    lof = glob.glob(mk)
    filehandler = open(lof[0], 'rb')
    masks_mem,masks_nuc,ref_mem = pkl.load(filehandler)
    print(len(ref_mem))
    filehandler.close()

    filehandler = open(ip, 'rb')
    img_stack = pkl.load(filehandler)
    filehandler.close()

    for img in img_stack[1:]:
        for i in range(img.shape[0]):
            img[i] *= 65535
            img[i] = img[i].astype(np.uint16)
            m = np.median(img[i][img[i] > 0])
            #print(np.percentile(img[i], (97.5, 99.8)))

#     flat_EPCAM = img_stack[0][0]
#     flat_yH2AX = img_stack[1][0].ravel()
#     flat_Ki67 = img_stack[2][0].ravel()
#     flat_cycA = img_stack[2][1].ravel()
#     flat_phoH3 = img_stack[2][2].ravel()
#     flat_cycB = img_stack[2][3].ravel()
#     flat_cPARP = img_stack[3][0].ravel()
#     flat_p21 = img_stack[3][1].ravel()
#     flat_p53 = img_stack[3][2].ravel()
#     # Populate both dictionaries
#     for key in ref_mem.keys(): 
#         mem_pxls = ref_mem[key]['Membrane Pixels 2D']
#         nuc_pxls = ref_mem[key]['Nuclei Pixels 2D']
#         nuc_size = len(ref_mem[key]["Nuclei Pixels"])
#         mem_size = len(ref_mem[key]["Membrane Pixels"])
#         if True:
#             sum_EPCAM_mem = np.sum(img_stack[0][0][mem_pxls])
#             sum_yH2AX_nuc = np.sum(img_stack[1][0][nuc_pxls])
#             sum_Ki67_nuc = np.sum(img_stack[2][0][nuc_pxls])
#             sum_cycB1_nuc = np.sum(img_stack[2][3][nuc_pxls])
#             sum_cycB1_mem = np.sum(img_stack[2][3][mem_pxls])
#             sum_phoH3_nuc = np.sum(img_stack[2][2][nuc_pxls])
#             sum_cycA_nuc = np.sum(img_stack[2][1][nuc_pxls])
#             sum_cPARP_nuc = np.sum(img_stack[3][0][nuc_pxls])
#             sum_p21_nuc = np.sum(img_stack[3][1][nuc_pxls])
#             sum_p53_nuc = np.sum(img_stack[3][2][nuc_pxls])
#             sum_p53_mem = np.sum(img_stack[3][2][mem_pxls])

#             # Get GFP Stats
#             ref_mem[key]['Avg EPCAM under Membrane'] = sum_EPCAM_mem/mem_size
#             ref_mem[key]['Avg yH2AX under Nuclei'] = sum_yH2AX_nuc/nuc_size
#             ref_mem[key]['Avg Ki67 under Nuclei'] = sum_Ki67_nuc/nuc_size
#             ref_mem[key]['Avg cycB1 under Membrane'] = sum_cycB1_mem/mem_size
#             ref_mem[key]['Avg cycB1 under Nuclei'] = sum_cycB1_nuc/nuc_size
#             ref_mem[key]['Avg phoH3 under Nuclei'] = sum_phoH3_nuc/nuc_size
#             ref_mem[key]['Avg cycA under Nuclei'] = sum_cycA_nuc/nuc_size
#             ref_mem[key]['Avg cPARP under Nuclei'] = sum_cPARP_nuc/nuc_size
#             ref_mem[key]['Avg p21 under Nuclei'] = sum_p21_nuc/nuc_size
#             ref_mem[key]['Avg p53 under Membrane'] = sum_p53_mem/mem_size
#             ref_mem[key]['Avg p53 under Nuclei'] = sum_p53_nuc/nuc_size

    filename = runName
#     rad_p2 = 107
#     rad_p98 = 110
#     rad = rescale_intensity(img_stack[0][1].astype('float64'), (rad_p2, rad_p98), (0, 255))
#     flat_rad = rad.ravel()
#     BRCA1_p2 = 109
#     BRCA1_p98 = 115
#     BRCA1 = rescale_intensity(img_stack[0][2].astype('float64'), (BRCA1_p2, BRCA1_p98), (0, 255))
#     flat_BRCA1 = BRCA1.ravel()
#     RPA2_p2 = 112
#     RPA2_p98 = 118
#     RPA2 = rescale_intensity(img_stack[0][3].astype('float64'), (RPA2_p2, RPA2_p98), (0, 255))
#     flat_RPA2 = RPA2.ravel()

#     rad51_spots = fociDetection(rad, masks_nuc, 'RAD51', filename + str(rad_p2) + '_' + str(rad_p98))
#     BRCA1_spots = fociDetection(BRCA1, masks_nuc, 'BRCA1', filename + str(BRCA1_p2) + '_' + str(BRCA1_p98))
#     rpa2_spots = fociDetection(RPA2, masks_nuc, 'RPA2', filename + str(RPA2_p2) + '_' + str(RPA2_p98))

    yH2AX_p2 = 108
    yH2AX_p98 = 125
    yH2AX_all = rescale_intensity(img_stack[1][0], (yH2AX_p2, yH2AX_p98), (0, 255))
    yH2AX_spots_all = fociDetection(yH2AX_all, masks_nuc, 'yH2AX', filename + str(yH2AX_p2) + '_' + str(yH2AX_p98))
    
    yH2AX_p2 = 130
    yH2AX_p98 = 160
    yH2AX_big = rescale_intensity(img_stack[1][0], (yH2AX_p2, yH2AX_p98), (0, 255))
    yH2AX_spots_big = fociDetection(yH2AX_big, masks_nuc, 'yH2AX', filename + str(yH2AX_p2) + '_' + str(yH2AX_p98))
#     flat_yH2AX = yH2AX.ravel()
    X53BP1_p2 = 105
    X53BP1_p98 = 110
    X53BP1_all = rescale_intensity(img_stack[1][1], (X53BP1_p2, X53BP1_p98), (0, 255))
    X53BP1_spots_all = fociDetection(X53BP1_all, masks_nuc, 'X53BP1', filename + str(X53BP1_p2) + '_' + str(X53BP1_p98))
#     flat_X53BP1 = X53BP1.ravel()
#     RAD18_p2 = 109
#     RAD18_p98 = 112
#     RAD18 = rescale_intensity(img_stack[1][2], (RAD18_p2, RAD18_p98), (0, 255))
#     flat_RAD18 = RAD18.ravel()

    X53BP1_p2 = 113
    X53BP1_p98 = 120
    X53BP1_big = rescale_intensity(img_stack[1][1], (X53BP1_p2, X53BP1_p98), (0, 255))
    X53BP1_spots_big = fociDetection(X53BP1_big, masks_nuc, 'X53BP1', filename + str(X53BP1_p2) + '_' + str(X53BP1_p98))
    
#     RAD18_spots = fociDetection(RAD18, masks_nuc, 'RAD18', filename + str(RAD18_p2) + '_' + str(RAD18_p98))

    # Strip the z axis off the detected foci coords and convert to integers
#     detectedFoci = [rad51_spots[:,0:2].astype('uint16'),
#                    BRCA1_spots[:,0:2].astype('uint16'), 
#                     rpa2_spots[:,0:2].astype('uint16'),
#                    yH2AX_spots[:,0:2].astype('uint16'),
#                    X53BP1_spots[:,0:2].astype('uint16'),
#                    RAD18_spots[:,0:2].astype('uint16')]
    detectedFoci = [yH2AX_spots_all[:,0:2].astype('uint16'),
                   yH2AX_spots_big[:,0:2].astype('uint16'),
                   X53BP1_spots_all[:,0:2].astype('uint16'),
                   X53BP1_spots_big[:,0:2].astype('uint16')]
    names = ('yH2AX_spots_all', 'yH2AX_spots_big', 'X53BP1_spots_all', 'X53BP1_spots_big')
#    names = ('RAD51 spots', 'BRCA1 spots', 'RPA2 spots', 'yH2AX spots','X53BP1 spots', 'RAD18 spots')

    # Create a dictionary where every key is a foci coordinate, and the value is the protein
    spotMatch = {}
    for i, spots in enumerate(detectedFoci):
        for spot in spots:
            if spotMatch.get((spot[0], spot[1])):
                spotMatch[(spot[0], spot[1])].append(names[i])
            else:
                spotMatch[(spot[0], spot[1])] = [names[i]]

    # For each nuclei, see which spots in the membrane are indexed in the key 
    for i, cellID in enumerate(ref_mem):
        for name in names:
            ref_mem[cellID][name] = 0

        # For each Nuclei Coordinate look for a spot
        nucCoord = numpyToList(ref_mem[cellID]['Nuclei Pixels 2D'])

        for coord in nucCoord:
            foci = spotMatch.get(coord)
            if foci:
                for focus in foci:
                    ref_mem[cellID][focus] += 1
                    
    filehandler = open(opdir+'ref_mem_foci_large&small_' + runName + '.pkl', 'wb')
    pkl.dump(ref_mem, filehandler)
    filehandler.close()
    
    # save CSV
    data = []
    for key in ref_mem.keys(): # use keys in the saved dict
        if True:
            data.append([key, 
                ref_mem[key]['yH2AX_spots_all'] ,
                ref_mem[key]['yH2AX_spots_big'] ,
                ref_mem[key]['X53BP1_spots_all'] ,
                ref_mem[key]['X53BP1_spots_big']])

    df2 = pd.DataFrame(data, columns=['cell_id',
                                     'yH2AX_spots_all',
                                     'yH2AX_spots_big',
                                     'X53BP1_spots_all',
                                     'X53BP1_spots_big'
                                     ])

    df2 = df2.sort_values(by="cell_id")
    df2.to_csv(opdir + runName +" _large&small_yH2AX" + '.csv', index=False)

P1_1
14793


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


Num yH2AX foci detected: 811294
Num yH2AX foci detected: 184790
Num X53BP1 foci detected: 483608
Num X53BP1 foci detected: 71464
P1_2
22485
Num yH2AX foci detected: 907375
Num yH2AX foci detected: 182349
Num X53BP1 foci detected: 568139
Num X53BP1 foci detected: 87517
P1_3
23416


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: invalid value encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:127: RuntimeWarning: invalid value encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)


Num yH2AX foci detected: 972544
Num yH2AX foci detected: 205664
Num X53BP1 foci detected: 554149
Num X53BP1 foci detected: 89990
P1_4
32449
Num yH2AX foci detected: 920877
Num yH2AX foci detected: 116392
Num X53BP1 foci detected: 562265
Num X53BP1 foci detected: 59657
P1_5
32921
Num yH2AX foci detected: 827019
Num yH2AX foci detected: 94424
Num X53BP1 foci detected: 465324
Num X53BP1 foci detected: 44507
P1_6
30321
Num yH2AX foci detected: 739934
Num yH2AX foci detected: 83190
Num X53BP1 foci detected: 223697
Num X53BP1 foci detected: 13815
P1_7
26627
Num yH2AX foci detected: 903841
Num yH2AX foci detected: 119585
Num X53BP1 foci detected: 197184
Num X53BP1 foci detected: 10771
P1_8
29549
Num yH2AX foci detected: 588580
Num yH2AX foci detected: 65858
Num X53BP1 foci detected: 439854
Num X53BP1 foci detected: 52114
P1_9
24517
Num yH2AX foci detected: 489781
Num yH2AX foci detected: 56292
Num X53BP1 foci detected: 421921
Num X53BP1 foci detected: 47576
P2_1
14339
Num yH2AX foci detected: